In [1]:
# Learning Rate 
# 너무 작거나 크면 발산하거나 Local minima가 생김 
# 일반적으로 0.001, 0.01 세팅하거나 loss를 봐서 설정해함

# Normalization (정규화)
# Min-Max Normaliztion
# 장점 : 0 ~ 1 사이로 모든 feature를 Scaling
# 단점 : 이상치에 취약

# Standardization (표준화)
# Z-Score Normalization 
# 장점 : 이상치에 덜 민감함
# 단점 : 동일한 척도로 Scaling 을 하지 않음

# Student's T분포

# underfitting => 너무 대충학습해서 학습이 잘 이루어지지 않은 경우
# overfitting => Training Data에 대해 너무 학습이 잘 된 경우 오히려 실제 데이터에 적용이 잘 안되는 케이스

# overfitting은 해결해야 하는 문제
# 1.  많은 양의 Training data
# 가장 기본이 되는 조건인데 우리가 program 적으로 해결이 불가능

# 2. Feature의 개수를 줄여야 해요!! 
# Titanic 예제에서 이름 같은 경우 종속변수와 연관이 없는 독랍변수들 제거 (상관관계 분석)

# 3. weight의 값을 인위적으로 조절!
# w 의 값이 클수로 curve를 그리게 되고 overfitting 의 여지가 많아져요
# Regulazation

# 4. Deep learning
# Dropout이라는 기능을 사용하여 overfitting 을 피할수 있어요

# 5. 학습의 수를 줄여요 ! 
# 적절한 학습방법의 수를 정해야합니다.
# 전체 학습데이터를 이용해서 1번학습 > 1 epoch / epoch수를 적절하게 조절



# 평가 (Evaluation)
# 우리가 가지고 있는 Training Data Set 을 어떻게 이용해서 성능평가를 할 것인가?
# 성능평가 방법은 (Metric)
# Precision 
# Recall 
# ** Accuracy

# 우리가 피해야하는 평가 방법 : Training Data Set 으로 학습한 후 
# Training Data Set으로 평가 진행 X 성능이 아주 좋게 나와요
# original Data Set
# Training Data
# Testing Data(최종 평가데이터) : 이 데이터 셋으로 Model을 수정하지 않아요!!

# 최종 모델의 Accuracy를 측정하기 위한 Data Set
# Training, Validation data set, testing
# Validation data set : Model의 개선작업 수행

# Evaluation 할때, 데이터가 충분하다면 !! Training, Validation, Testing
# 데이터량이 작으면 underfitting epoch의 수를 늘리면 overfitting이 발생
# cross validation (cv) - K-fold 
# K :상수값 만약 K=5 이면 fold를 5개로 나눔 5개로 나누고 테스트와 Training을 반복



In [1]:
# Multinomial Classfication 
# BMI 지수에 대한 데이터를 가지고 학습 후 예측까지 진행해 볼꺼에요!!
# BMI 지수는 키와 몸무게를 가지고 저체중, 정상, 과체중, 비만을 판단하는 지수
# BMI = 자신의 몸무게(kg) / 키의 제곱(m)
# 18.5 이하면 저체중
# 18.5 ~23 정상
# 23 ~ 25 과체중
# 25이상 비만

# 우리가 하려는건 식이 아니라 BMI지수를 조사한 데이터가 있어요
# 이걸 학습해서 예측을 통해 나의 BMI지수를 알아보려고 합니다.
# 단, 제공하는 데이터는 4가지 분류가 아니라 3가지 분류로 나누어져 있어요!

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score


# Raw Data Loading
df = pd.read_csv('./data/bmi.csv', skiprows=3)
display(df)
# 결측치부터 확인!
# df.isnull().sum()

# 이상치확인
# Z-score

zscore_threshold = 1.8

for col in df.columns:
    outlier = df[col][np.abs(stats.zscore(df[col])) > zscore_threshold]    
    df = df.loc[~df[col].isin(outlier)]
    
    
df.loc[np.abs(stats.zscore(df['height']))>=zscore_threshold,:] # height
df.loc[np.abs(stats.zscore(df['weight']))>=zscore_threshold,:] # weight
df.loc[np.abs(stats.zscore(df['label']))>=zscore_threshold,:] # label

# Data Split
# Train, Test 두 부분으로 분할
# 분리하는 비율은 7:3 으로 분리.
# 나중에 Train 부분은 K-fold cross validation을 진행할 꺼에요!

# 역슬레시로 칸넘기기
x_data_train, x_data_test, t_data_train, t_data_test =\
train_test_split(df[['height', 'weight']], df['label'], test_size=0.3, random_state=0) # 30% 테스트 사이즈

display(x_data_train)
display(x_data_test)

#Normalization
scaler = MinMaxScaler() # scaler 객체를 생성해요!
scaler.fit(x_data_train) # scaler 객체에 최대, 최소와 같은 정보가 들어가요!
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train #혼동을 방지하기 위해서 변수를 삭제
del x_data_test

# sklearn 구현은 매우매우 간단 !!
model = LogisticRegression()
model.fit(x_data_train_norm, t_data_train)

# 우리 model의 정확도를 측정해야 해요!
# cross validation
kfold = 10 
kfold_score = cross_val_score(model, x_data_train_norm, t_data_train, cv = kfold)
print('##########cross validation############')

print('score : {}'.format(kfold_score))
print('전체평균은 : {}'.format(kfold_score.mean()))

# 최종모델평가
predict_val = model.predict(x_data_test_norm) #테스트데이터로 예측값을 구해요!
acc = accuracy_score(predict_val, t_data_test)
print('우리 model의 최종 Accuracy : {}'.format(acc))

# Predict
height = 188
weight = 78
my_state = [[height, weight]]
my_state_val = model.predict(scaler.transform(my_state))

print(my_state_val)

,label,height,weight
0,1,188,71
1,2,161,68
2,0,178,52
3,2,136,63
4,1,145,52
...,...,...,...
19995,0,163,48
19996,2,139,70
19997,1,150,48
19998,1,189,69


,height,weight
8226,195,71
9398,120,74
11646,121,61
12193,158,36
18601,182,78
...,...,...
13123,137,60
19648,193,61
9845,146,79
10799,175,68


,height,weight
19134,163,44
4981,153,66
16643,180,38
19117,171,61
5306,173,40
...,...,...
9532,196,47
9764,145,43
19241,178,55
3452,156,38


##########cross validation############
score : [0.98       0.98642857 0.985      0.97642857 0.98642857 0.98428571
 0.98714286 0.97714286 0.97714286 0.98642857]
전체평균은 : 0.9826428571428572
우리 model의 최종 Accuracy : 0.9845
[1]
